In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
def create_sequences(df, feature_cols, target_col, window):
    X, y = [], []

    arr = df[feature_cols].values      # convert ONCE
    target_arr = df[target_col].values # convert ONCE

    for i in range(len(df) - window):
        X.append(arr[i:i+window])
        y.append(target_arr[i+window])

    return np.array(X), np.array(y)


In [4]:
import pandas as pd
df = pd.read_csv("E://PHD/Course materials/Sem 1/Data Mining/Assignment/workspace/stock_price_trend_prediction/stock_price_pred_data_mining/Data/05_cluster_output/cluster_output_Kmeans.csv")
# 1. Compute next-week (5 trading days) forward return per stock
df['next_week_close'] = df.groupby('stock_id')['Close'].shift(-5)
df['next_week_return'] = (df['next_week_close'] - df['Close']) / df['Close']
# 2. Binary target: 1 = uptrend, 0 = downtrend or flat
df["target"] = (df["Close"].shift(-5) > df["Close"]).astype(int)
# 3. Drop rows where we can't compute future return (last 5 days per stock)
df = df.dropna(subset=['next_week_return'])
# Optional: drop helper column
df = df.drop(columns=['next_week_close'])
drop_cols = ['Date', 'stock_id', 'cluster', 'target', 'next_week_return']
feature_cols = [c for c in df.columns if c not in drop_cols]

print("Number of features:", len(feature_cols))
print("Features:", feature_cols)

Number of features: 23
Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'MA7', 'MA21', 'EMA20', 'EMA50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower', 'CCI_14', 'CMF_20', 'Stoch_K', 'Stoch_D', 'Momentum_10', 'Daily_Return', 'Log_Return']


In [5]:
df.shape

(72548, 28)

In [ ]:
WINDOW = 20  # recommended
cluster_lstm_models = {}

for cl in df['cluster'].unique():

    print(f"\nTraining LSTM for Cluster {cl}")
    cluster_df = df[df['cluster'] == cl].copy().sort_values(["stock_id", "Date"])
    
    X_all, y_all = [], []

    # ---- SEQUENCE BUILDING PER STOCK (memory-safe)
    for sid, temp in cluster_df.groupby("stock_id"):
        if len(temp) <= WINDOW + 2:
            continue
        X_temp, y_temp = create_sequences(temp, feature_cols, 'target', WINDOW)
        X_all.append(X_temp)
        y_all.append(y_temp)

    if not X_all:
        print(f"No usable data for cluster {cl}")
        continue

    # Combine results
    X = np.vstack(X_all)
    y = np.concatenate(y_all)

    print(f"Cluster {cl}: Final sequence shape = {X.shape}")

    # ---- Train-test split (time-based)
    split_idx = int(0.7 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # ---- Build LSTM Model
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=(WINDOW, len(feature_cols))),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # ---- Fit model
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=30,
        batch_size=128,
        callbacks=[es],
        verbose=1
    )

    # ---- Store model
    cluster_lstm_models[cl] = model

    # ---- Evaluate
    y_pred_prob = model.predict(X_test).flatten()
    y_pred = (y_pred_prob > 0.5).astype(int)

    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec  = recall_score(y_test, y_pred, zero_division=0)
    f1   = f1_score(y_test, y_pred, zero_division=0)

    from sklearn.metrics import roc_auc_score

    try:
        rocauc = roc_auc_score(y_test, y_pred_prob)
    except:
        rocauc = 0.0   # when only 1 class in test set

    print(f"ROC-AUC: {rocauc:.4f}")


    print(f"""
    🔹 Cluster {cl} Evaluation:
       Accuracy:  {acc:.4f}
       Precision: {prec:.4f}
       Recall:    {rec:.4f}
       Macro-F1:  {f1:.4f}
    """)



🔵 Training LSTM for Cluster 1
Cluster 1: Final sequence shape = (55959, 20, 23)
Epoch 1/30


c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


245/245 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5089 - loss: 0.7005 - val_accuracy: 0.5034 - val_loss: 0.6943
Epoch 2/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5139 - loss: 0.6940 - val_accuracy: 0.5093 - val_loss: 0.6945
Epoch 3/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5210 - loss: 0.6926 - val_accuracy: 0.5063 - val_loss: 0.6938
Epoch 4/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5228 - loss: 0.6920 - val_accuracy: 0.5084 - val_loss: 0.6930
Epoch 5/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5236 - loss: 0.6921 - val_accuracy: 0.5086 - val_loss: 0.6934
Epoch 6/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5227 - loss: 0.6921 - val_accuracy: 0.5104 - val_loss: 0.6929
Epoch 7/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5252 - loss: 0.6920 - val_accuracy: 0.5103 - val_loss: 0.6930
Epoch 8/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5243 - loss: 0.6920 - val_accuracy: 0.5101 - val_

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5086 - loss: 0.7152 - val_accuracy: 0.5471 - val_loss: 0.6888
Epoch 2/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5104 - loss: 0.7042 - val_accuracy: 0.5471 - val_loss: 0.6906
Epoch 3/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5192 - loss: 0.6964 - val_accuracy: 0.5471 - val_loss: 0.6891
Epoch 4/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5226 - loss: 0.6950 - val_accuracy: 0.5471 - val_loss: 0.6891
Epoch 5/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5319 - loss: 0.6927 - val_accuracy: 0.5471 - val_loss: 0.6890
Epoch 6/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5271 - loss: 0.6915 - val_accuracy: 0.5471 - val_loss: 0.6883
Epoch 7/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5352 - loss: 0.6918 - val_accuracy: 0.5471 - val_loss: 0.6883
Epoch 8/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5341 - loss: 0.6914 - val_accuracy: 0.5471 - val_loss: 0.6881
Epo